# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse     Shipments               Customer  \
0  Mar 21 2023  1:53PM  259661        19   ADV80014272    AdvaGen Pharma, Ltd   
1  Mar 21 2023  1:33PM  259660        19       9291006            Purify Life   
2  Mar 21 2023  1:25PM  259659        15      30014876  Alliance Pharma, Inc.   
3  Mar 21 2023  1:25PM  259659        15  ALI039379656  Alliance Pharma, Inc.   
4  Mar 21 2023  1:25PM  259659        15      30014877  Alliance Pharma, Inc.   
5  Mar 21 2023  1:25PM  259659        15      30014878  Alliance Pharma, Inc.   
6  Mar 21 2023  1:25PM  259659        15      30014879  Alliance Pharma, Inc.   
7  Mar 21 2023  1:25PM  259659        15      30014880  Alliance Pharma, Inc.   
8  Mar 21 2023  1:25PM  259659        15      30014893  Alliance Pharma, Inc.   
9  Mar 21 2023  1:25PM  259659        15      30014894  Alliance Pharma, Inc.   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4      Executing  
5      Executing  
6      Executing  
7      Executing  
8      Executing  
9      Executing

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
28  259658       Released          1
29  259659      Executing         18
30  259659       Released          2
31  259660       Released          1
32  259661       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
259657                NaN        1.0       NaN
259658                NaN        NaN       1.0
259659                NaN       18.0       2.0
259660                NaN        NaN       1.0
259661                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259596                0.0        1.0       0.0
259599                0.0        2.0       0.0
259606               35.0       29.0       9.0
259610                0.0        1.0       1.0
259613                0.0        0.0       1.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
259596                  0          1         0
259599                  0          2         0
259606                 35         29         9
259610                  0          1         1
259613                  0          0         1

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               259596          0          1         0
1               259599          0          2         0
2               259606         35         29         9
3               259610          0          1         1
4               259613          0          0         1

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               259596                   1         
1               259599                   2         
2               259606        35        29        9
3               259610                   1        1
4               259613                            1

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd
1   Mar 21 2023  1:33PM  259660        19            Purify Life
2   Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.
22  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.
23  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.
24  Mar 21 2023  1:12PM  259656        19  VITABIOTICS  USA INC.
25  Mar 21 2023  1:03PM  259655        10      Yusen – 3D Matrix
26  Mar 21 2023 12:57PM  259652        10       Methapharm, Inc.
33  Mar 21 2023 12:57PM  259653        10            Emerginnova
34  Mar 21 2023 12:54PM  259650        10              Hush Hush

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd             
1  Mar 21 2023  1:33PM  259660        19            Purify Life             
2  Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.             
3  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.             
4  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.             
5  Mar 21 2023  1:12PM  259656        19  VITABIOTICS  USA INC.             
6  Mar 21 2023  1:03PM  259655        10      Yusen – 3D Matrix             
7  Mar 21 2023 12:57PM  259652        10       Methapharm, Inc.             
8  Mar 21 2023 12:57PM  259653        10            Emerginnova             
9  Mar 21 2023 12:54PM  259650        10              Hush Hush             

  Executing Released  
0                  1  
1                  1  
2        18        2  
3                  1  
4         1           
5         1           
6                  1  
7         3        4  
8                  1  
9                  6

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd        1   
1  Mar 21 2023  1:33PM  259660        19            Purify Life        1   
2  Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.        2   
3  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.        1   
4  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.            
5  Mar 21 2023  1:12PM  259656        19  VITABIOTICS  USA INC.            
6  Mar 21 2023  1:03PM  259655        10      Yusen – 3D Matrix        1   
7  Mar 21 2023 12:57PM  259652        10       Methapharm, Inc.        4   
8  Mar 21 2023 12:57PM  259653        10            Emerginnova        1   
9  Mar 21 2023 12:54PM  259650        10              Hush Hush        6   

  Executing Completed  
0                      
1                      
2        18            
3                      
4         1            
5         1            
6                      
7         3            
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd        1   
1  Mar 21 2023  1:33PM  259660        19            Purify Life        1   
2  Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.        2   
3  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.        1   
4  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.            

  Executing Completed  
0                      
1                      
2        18            
3                      
4         1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd       1.0   
1  Mar 21 2023  1:33PM  259660        19            Purify Life       1.0   
2  Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.       2.0   
3  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.       1.0   
4  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.       NaN   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2       18.0        NaN  
3        NaN        NaN  
4        1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Mar 21 2023  1:53PM  259661        19    AdvaGen Pharma, Ltd       1.0   
1  Mar 21 2023  1:33PM  259660        19            Purify Life       1.0   
2  Mar 21 2023  1:25PM  259659        15  Alliance Pharma, Inc.       2.0   
3  Mar 21 2023  1:17PM  259658        22      NBTY Global, Inc.       1.0   
4  Mar 21 2023  1:16PM  259657        22      NBTY Global, Inc.       0.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2       18.0        0.0  
3        0.0        0.0  
4        1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3115621      36.0       22.0        4.0
15          259659       2.0       18.0        0.0
19         2596375      15.0       33.0       35.0
20          259623       0.0        9.0       23.0
22          519315       1.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3115621      36.0       22.0        4.0
1        15   259659       2.0       18.0        0.0
2        19  2596375      15.0       33.0       35.0
3        20   259623       0.0        9.0       23.0
4        22   519315       1.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      36.0       22.0        4.0
1        15       2.0       18.0        0.0
2        19      15.0       33.0       35.0
3        20       0.0        9.0       23.0
4        22       1.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   36.0
1        15  Released    2.0
2        19  Released   15.0
3        20  Released    0.0
4        22  Released    1.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    19    20   22
Status                                
Completed   4.0   0.0  35.0  23.0  0.0
Executing  22.0  18.0  33.0   9.0  1.0
Released   36.0   2.0  15.0   0.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    19    20   22
0          Completed   4.0   0.0  35.0  23.0  0.0
1          Executing  22.0  18.0  33.0   9.0  1.0
2           Released  36.0   2.0  15.0   0.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    19    20   22
0  Completed   4.0   0.0  35.0  23.0  0.0
1  Executing  22.0  18.0  33.0   9.0  1.0
2   Released  36.0   2.0  15.0   0.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()